# 9강. Spark cluster & word count

# Map Reduce용 Spark API 실습

### key와 value로 데이터를 구분해보자
- Spark는 주어진 Rdd의 row가 가지고 있는 첫번째 column을 key로 인식함
- 대부분의 Key가 필요한 함수들은 접미어로 Key를 가지고 있음
 + ex> sortByKey, reduceByKey

In [1]:
import findspark
findspark.init('/opt/homebrew/Cellar/apache-spark/3.5.1/libexec')
import pyspark
sc = pyspark.SparkContext(appName="myAppName")

24/04/03 19:24:10 WARN Utils: Your hostname, itaehun-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.24.96.75 instead (on interface en0)
24/04/03 19:24:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 19:24:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<div class="alert alert-warning"/>
- map<br>
 + 주어진 rdd에 func로 맵 변형을 수행한다<br><br>
 
- flatMap<br>
 + 주어진 rdd에 func으로 맵 변형을 주고 배열을 하나 벗겨 flatten한다<br><br>
- reduceByKey<br>
 + 주어진 rdd를 key를 기준으로 collect후 주어진 func을 수행<br>

In [4]:
#중첩된 형태의 rdd를 하나의 row 단위로 풀어 줄 수 있는 명령, 대단히 중요한 명령
rdd = sc.parallelize([(1, 2), (1, 3), (1, 2), (2, 4)]) # 앞은 key, 뒤는 value
print(rdd.flatMap(lambda x:x).collect())
print()

rdd = sc.parallelize([(1, 2), (1, 3), (1, 2), (2, 4)])
print(rdd.map(lambda x:x).collect())
print()

#중첩된 정도에 따라 row로 풀리는 단위가 다름
rdd = sc.parallelize([((1, 2), (1, 3)), ((1, 2), (2, 4))])
print(rdd.flatMap(lambda x:x).flatMap(lambda x:x).collect())

[1, 2, 1, 3, 1, 2, 2, 4]

[(1, 2), (1, 3), (1, 2), (2, 4)]

[1, 2, 1, 3, 1, 2, 2, 4]


In [7]:
rdd = sc.parallelize(["a", "b", "c,d", "e,f"])
rdd.flatMap(lambda x:x).collect()
# rdd.map(lambda x:x).collect()

['a', 'b', 'c', ',', 'd', 'e', ',', 'f']

In [8]:
rdd.flatMap(lambda x:x.split(",")).collect()

['a', 'b', 'c', 'd', 'e', 'f']

In [10]:
rdd.flatMap(lambda x:x.split(",")).collect()

['a', 'b', 'cd', 'ef']

In [11]:
#reduceByKey를 이용하여 key 별로 reduce를 수행할 수 있음
rdd = sc.parallelize([(1, 2), (1, 3), (1, 2), (2, 4), (1, 5), (3,2), (3,5), (10,1), (10, 2)])

#1 -> (1,2) (1,3) ( 1,2) (1,5) => [2, 3, 2, 5]
#2 -> (2,4) => [4]
#3 -> (3,2) (3,5) => [2, 5]
#10 -> (10,1) (10,2) => [1, 2]
rdd.reduceByKey(lambda a,b: a+b).collect()

[(1, 12), (2, 4), (10, 3), (3, 7)]

In [12]:
#groupBy와 유사한 groupByKey동작도 존재함
#reduceByKey는 함수로 처리된 하나의 결과값을 남겨두지만
#groupByKey는 모든 원소를 보유하고 순회할 수 있음
rdd = sc.parallelize([(1, 2), (1, 3), (1, 2), (2, 4), (1, 5)])
grouped = rdd.groupByKey() # 메모리를 많이 잡아먹기 때문에 reduceByKey를 사용함
print(grouped.collect())

#grouped data중 1이 key인 group 데이터를 추출
a, b = grouped.collect()
print(a[0], list(a[1]))

[(1, <pyspark.resultiterable.ResultIterable object at 0x1095f6940>), (2, <pyspark.resultiterable.ResultIterable object at 0x1095223a0>)]
1 [2, 3, 2, 5]


In [13]:
#key를 group 기준으로 임의로 만들어 준 뒤 groupByKey나 reduceByKey를 돌리는 것이 일반적
#다음 데이터에 짝수, 홀수의 기준으로 key를 분리한뒤 groupByKey로 각 원소를 출력해보자
rdd = sc.parallelize([1,2,3,4,5,6,7,8])
rdd = rdd.map(lambda x:(x % 2 ==0, x))
print ( rdd.groupByKey().collect() )

a, b = rdd.groupByKey().collect()
print (list(a[1]))
print (list(b[1]))

[(False, <pyspark.resultiterable.ResultIterable object at 0x1096118b0>), (True, <pyspark.resultiterable.ResultIterable object at 0x109522e50>)]
[1, 3, 5, 7]
[2, 4, 6, 8]


<div class="alert alert-warning"/>
- mapValues(func)<br>
 + 주어진 rdd를 순회하면서 key와 value 중 value 부분에 func을 적용<br><br>
- flatMapValues(func)<br>
 + 주어진 rdd를 flat 동작을 value 기준으로 수행하면서 key와 합쳐서 생성해냄<br><br>


In [14]:
#mapValues는 key, value로 나눠진 데이터에 대하여 value 쪽만 function으로 컨트롤 가능
rdd = sc.parallelize(["a", "b", "c"])
rdd = rdd.map(lambda x: (x, 0)) # (a, 0), (b, 0), (c, 0)
rdd = rdd.mapValues(lambda x:x+1) # ((a, 1), (b, 1), (c, 1))
rdd.collect()

[('a', 1), ('b', 1), ('c', 1)]

In [15]:
a = sc.parallelize(["a", "b", "c"]).map(lambda x: (x, 0))  # ('a', 0), ('b', 0), ('c', 0)
b = a.map(lambda x: (x[0], x[1]+1)) # ('a', 1) ('b', 1)
b.collect()

[('a', 1), ('b', 1), ('c', 1)]

In [16]:
#flatMapValues를 이용하여 key에 대하여 각 value들을 모두 새로운 기준의 row로 분리 가능
print( rdd.mapValues(lambda x:(x, x)).collect() )
print( rdd.mapValues(lambda x:(x, x)).flatMapValues(lambda x:x).collect() )

[('a', (1, 1)), ('b', (1, 1)), ('c', (1, 1))]
[('a', 1), ('a', 1), ('b', 1), ('b', 1), ('c', 1), ('c', 1)]


### 위키 피디아 파일을 읽어보자
- Spark는 plain text에 대한 read/write 및 rdd 전환을 지원함

In [18]:
import os
print ( os.getcwd() )

/Users/itaehun/python/빅실무


In [19]:
with open('./data/wiki_wordcount.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[-3:]:
        print (line.strip())

This page was last edited on 11 June 2018, at 19:08 (UTC).
Text is available under the Creative Commons Attribution-ShareAlike License; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia® is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.
Privacy policyAbout WikipediaDisclaimersContact WikipediaDevelopersCookie statementMobile viewWikimedia Foundation Powered by MediaWiki


<div class="alert alert-warning"/>
- sparkContext.textFile(filePath, partitionNum = None)<br>
 + 파일을 읽어옴, 주어진 개수로 파티션을 분리할 수 있음
 + wildcard 및 정규 표현식 사용 가능<br><br>
- rdd.saveAsTextFile(path, compressionCodecClass=None)<br>
 + 주어진 rdd를 plain text로 저장함, compressionCodecClass를 통하여 압축방법을 지정가능<br>

In [20]:
#wild card를 지정하여 파일을 읽을 수 있음
print(type(sc.textFile('./data/wiki_wordcount.txt')))
print(sc.textFile('./data/wiki_wordcount.txt').getNumPartitions())
print(sc.textFile('./data/wiki_*.txt', 3).getNumPartitions())

<class 'pyspark.rdd.RDD'>
2
3


In [21]:
sc.textFile('./data/wiki_wordcount.txt').collect()

['',
 'Word count',
 'From Wikipedia, the free encyclopedia',
 'Jump to navigationJump to search',
 'The word count is the number of words in a document or passage of text. Word counting may be needed when a text is required to stay within certain numbers of words. This may particularly be the case in academia, legal proceedings, journalism and advertising. Word count is commonly used by translators to determine the price for the translation job. Word counts may also be used to calculate measures of readability and to measure typing and reading speeds (usually in words per minute). When converting character counts to words, a measure of 5 or 6 characters to a word is generally used for English.[1]',
 '',
 '',
 'Contents',
 '1\tDetails and variations of definition',
 '2\tSoftware',
 '3\tIn fiction',
 '4\tIn non-fiction',
 '5\tSee also',
 '6\tReferences',
 '7\tSources',
 'Details and variations of definition',
 '',
 'This section does not cite any sources. Please help improve this sectio

### word count를 작성하여 보자
- 1. text파일을 rdd로 변환
- 2. map을 이용하여 word 기준으로 분리
- 3. flatMap을 이용하여 한 단어기준으로 row 추출
- 4. 각 word에 1이라는 count를 value에 부여
- 5. reduce를 실행하여 각 word 별 count를 집계

In [25]:
#map 부분을 완성해보자
rdd = sc.textFile('./data/wiki_wordcount.txt')
rdd1 = rdd.map(lambda line:line.split(' '))
rdd2 = rdd1.flatMap(lambda x:x)
rdd3 = rdd2.map(lambda word:(word, 1))
rdd3.collect()[:10]

[('', 1),
 ('Word', 1),
 ('count', 1),
 ('From', 1),
 ('Wikipedia,', 1),
 ('the', 1),
 ('free', 1),
 ('encyclopedia', 1),
 ('Jump', 1),
 ('to', 1)]

In [26]:
#reduce 부분을 완성한 후 10개를 출력
rdd4 = rdd3.reduceByKey(lambda x,y:x+y)
rdd4.top(10)

[('فارسی', 1),
 ('you', 1),
 ('writers', 1),
 ('writer,[7]', 1),
 ('writer', 1),
 ('write', 1),
 ('workers)', 1),
 ('work', 1),
 ('words.[9]', 1),
 ('words.', 3)]

In [27]:
#결과를 저장
rdd4.saveAsTextFile('./data/word_count_result')

### 기존에 사용했던 rdd를 빠르게 재사용해보자
- 일반적으로 rdd는 action이 수행될때 관련된 transformation 동작을 다시 수행함
- 적은 데이터와 짧은 lineage를 가지고 있는 rdd는 결과가 빠르게 나오지만 데이터가 크거나 lineage가 크다면 결과까지 상당한 시간이 소요 될 수 있음
- 2번 이상의 재사용성을 가진 rdd는 memory나 storage에 저장하고 다시 재활용이 가능함

<div class="alert alert-warning"/>
- rdd.persist(storageLevel)<br>
 + rdd를 memory나 hdd에 저장함<br>
 + storageLevel은 pyspark.StorageLevel에서 결정할 수 있음<br><br>
- rdd.unpersist()<br>
 + 저장되었던 rdd를 memory나 hdd에서 해제함
- rdd.cache()<br>
 + rdd 결과를 memory에 저장함, rdd.persist(pyspark.StorageLevel.MEMORY_ONLY)와 같은 동작

In [28]:
rdd = sc.parallelize(range(5000000))
rdd1 = rdd.map(lambda x:(x%13, 1))
rdd2 = rdd1.reduceByKey(lambda a,b:a+b)
rdd2.top(3)

[(12, 384615), (11, 384615), (10, 384615)]

In [29]:
rdd2.top(3)

[(12, 384615), (11, 384615), (10, 384615)]

In [30]:
rdd_cache = rdd2.cache()
rdd_persist = rdd2.persist(pyspark.StorageLevel.MEMORY_ONLY)

In [31]:
rdd_cache.top(3)

[(12, 384615), (11, 384615), (10, 384615)]

In [32]:
rdd_persist.top(3)

[(12, 384615), (11, 384615), (10, 384615)]

In [33]:
rdd_cache = rdd_cache.unpersist()
rdd_persist = rdd_persist.unpersist()

24/04/03 20:19:35 WARN BlockManager: Asked to remove block rdd_89_5, which does not exist
